In [16]:
import pandas as pd

# df = pd.read_csv('../res/crawl_investor_daily_volume.csv').fillna(0)
df = pd.read_csv('../res/crwal_investor_stock_type.csv')
df = df.sort_values(by=['DATEON'], axis=0, ascending=True)
df

,TYPE_CODE,DATEON,END_VALUE,DELTA,UPDOWN_PER,VOLUME,COMP_BUY,FOR_BUY,FOR_CONT,FOR_PER
0,233740,2016.01.13,11570.0,520.0,4.71,1750580.0,-41337.0,0.0,0.0,0.00
1,233740,2016.01.14,11445.0,-125.0,-1.08,1795910.0,55991.0,0.0,0.0,0.00
2,233740,2016.01.15,11230.0,-215.0,-1.88,2078430.0,-210233.0,0.0,0.0,0.00
3,233740,2016.01.18,11575.0,345.0,3.07,1900820.0,89002.0,0.0,0.0,0.00
4,233740,2016.01.19,11550.0,-25.0,-0.22,2014000.0,166403.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...
2095,233740,2024.07.22,10210.0,-530.0,-4.93,25376100.0,-10601000.0,-671720.0,50339.0,0.05
2096,233740,2024.07.23,10275.0,65.0,0.64,20878400.0,-990217.0,192766.0,343105.0,0.32
2097,233740,2024.07.24,10360.0,85.0,0.83,18236500.0,1033550.0,160263.0,603368.0,0.55
2098,233740,2024.07.25,9935.0,-425.0,-4.10,27670600.0,-8731450.0,-560720.0,42648.0,0.04


In [36]:
df['FOR_BUY_D14']=df.FOR_BUY.rolling(14).sum().shift(1)
df['FOR_BUY_D7']=df.FOR_BUY.rolling(3).sum().shift(1)
df['COMP_BUY_D14']=df.COMP_BUY.rolling(14).sum().shift(1)
df['COMP_BUY_D7']=df.COMP_BUY.rolling(3).sum().shift(1)
df['DELTA_OUT']=df.END_VALUE.shift(-1)

df = df.sort_values(by=['DATEON'], axis=0, ascending=True)
df = df[['DATEON', 'END_VALUE', 'DELTA_OUT', 'VOLUME', 'UPDOWN_PER', 'COMP_BUY', 'COMP_BUY_D7', 'COMP_BUY_D14', 'FOR_BUY', 'FOR_BUY_D7', 'FOR_BUY_D14']]

df.dropna()

,DATEON,END_VALUE,DELTA_OUT,VOLUME,UPDOWN_PER,COMP_BUY,COMP_BUY_D7,COMP_BUY_D14,FOR_BUY,FOR_BUY_D7,FOR_BUY_D14
14,2016.02.02,11985.0,11870.0,1091070.0,0.33,66242.0,399455.0,763967.0,0.0,0.0,0.0
15,2016.02.03,11870.0,11875.0,1198020.0,-0.96,-46529.0,397148.0,871546.0,0.0,0.0,0.0
16,2016.02.04,11875.0,11795.0,1816400.0,0.04,-50894.0,329028.0,769026.0,0.0,0.0,0.0
17,2016.02.05,11795.0,10645.0,1017640.0,-0.67,-8125.0,-31181.0,928365.0,0.0,0.0,0.0
18,2016.02.11,10645.0,9100.0,1942020.0,-9.75,-678720.0,-105548.0,831238.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2094,2024.07.19,10740.0,10210.0,15638800.0,1.61,-296311.0,-16052730.0,-1021700.0,46680.0,16533.0,154128.0
2095,2024.07.22,10210.0,10275.0,25376100.0,-4.93,-10601000.0,-10529091.0,-4560341.0,-671720.0,-126975.0,291888.0
2096,2024.07.23,10275.0,10360.0,20878400.0,0.64,-990217.0,-16269441.0,-5561821.0,192766.0,-764412.0,151081.0
2097,2024.07.24,10360.0,9935.0,18236500.0,0.83,1033550.0,-11887528.0,-13337018.0,160263.0,-432274.0,-643820.0


In [37]:
train = df[df['DATEON'] < '2024.05.01']
test = df.drop(train.index)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

train = train.drop(columns=['DATEON'])
test = test.drop(columns=['DATEON'])

In [38]:
from pycaret.regression import *

s = setup(data = train, target='DELTA_OUT', session_id=123, normalize=True, normalize_method='minmax')

,Description,Value
0,Session id,123
1,Target,DELTA_OUT
2,Target type,Regression
3,Original data shape,"(2041, 10)"
4,Transformed data shape,"(2041, 10)"
5,Transformed train set shape,"(1428, 10)"
6,Transformed test set shape,"(613, 10)"
7,Numeric features,9
8,Rows with missing values,0.7%
9,Preprocess,True


In [39]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:20:46
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,287.7892,176117.5227,415.5433,0.9890,0.0332,0.0244,0.0050
llar,Lasso Least Angle Regression,287.7892,176117.5453,415.5434,0.9890,0.0332,0.0244,0.0040
omp,Orthogonal Matching Pursuit,288.2188,176688.9320,416.2785,0.9890,0.0332,0.0245,0.0040
huber,Huber Regressor,288.4395,177150.5059,416.4466,0.9890,0.0335,0.0246,0.0070
lr,Linear Regression,289.3689,177703.6203,417.1502,0.9889,0.0336,0.0246,0.0200
lar,Least Angle Regression,289.3688,177703.6211,417.1502,0.9889,0.0336,0.0246,0.0040
br,Bayesian Ridge,289.3768,177700.3383,417.1497,0.9889,0.0336,0.0246,0.0040
par,Passive Aggressive Regressor,293.0438,182932.4391,423.7201,0.9886,0.0343,0.0251,0.0060
ridge,Ridge Regression,304.4248,189753.5727,432.3328,0.9882,0.0348,0.0260,0.0050
gbr,Gradient Boosting Regressor,308.7811,205365.9291,449.2801,0.9871,0.0349,0.0258,0.0530
